<style>
*
{
	text-align: justify;
	line-height: 1.5;
	font-family: "Arial", sans-serif;
	font-size: 12px;
}

h2, h3, h4, h5, h6
{
	font-family: "Arial", sans-serif;
	font-size: 12px;
	font-weight: bold;
}
h2
{
	font-size: 14px;
}
h1
{
	font-family: "Wingdings", sans-serif;
	font-size: 16px;
}
</style>

## EDA of Irish agriculture output, input, income values at current prices.

<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("tahirawwad", "agriculture-data-analytics", "notebooks/notebook-2-03-eda-irish-milk-production.ipynb", "master"))
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/tahirawwad/agriculture-data-analytics/master?filepath=notebooks/notebook-2-03-eda-irish-milk-production.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/tahirawwad/agriculture-data-analytics/blob/master/notebooks/notebook-2-03-eda-irish-milk-production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

### Objective

The objective is to provide an Exploratory Data Analysis (EDA) of the [cso-aea01-value-at-current-prices-for-output-input-and-income-in-agriculture.csv](./../assets/cso-aea01-value-at-current-prices-for-output-input-and-income-in-agriculture.csv) file provided by the <a href="https://data.cso.ie/table/AEA01" target="_new">CSO: AEA01 Table</a>. The EDA is performed to investigate and clean the data, to spot anomalies.  

### Setup

Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
# Local
#!pip install -r script/requirements.txt
# Remote option
#!pip install -r https://raw.githubusercontent.com/tahirawwad/agriculture-data-analytics/requirements.txt
#Options: --quiet --user

In [2]:
from agriculture_data_analytics.project_manager import *
from agriculture_data_analytics.dataframe_labels import *
from pandas import DataFrame
import data_analytics.exploratory_data_analysis as eda
import data_analytics.exploratory_data_analysis_reports as eda_reports
import data_analytics.github as github
import os
import pandas

In [3]:
artifact_manager: ProjectArtifactManager = ProjectArtifactManager()
asset_manager: ProjectAssetManager = ProjectAssetManager()
artifact_manager.is_remote = asset_manager.is_remote = True
github.display_jupyter_notebook_data_sources(
    [asset_manager.get_bovine_tuberculosis_filepath()])
artifact_manager.is_remote = asset_manager.is_remote = False

https://github.com/markcrowe-com/agriculture-data-analytics/assets/cso-daa01-bovine-tuberculosis-2022-01-Jan-15.csv?raw=true


### Loading the CSV file

#### Create Data Frames

In [4]:
filepath: str = './../assets/cso-aea01-value-at-current-prices-for-output-input-and-income-in-agriculture.csv'
agriculture_prices_dataframe: DataFrame = pandas.read_csv(filepath)

#### Renaming Columns

In [5]:
old_to_new_column_names_dictionary = {
    UNIT.upper(): UNIT,
    VALUE.upper(): VALUE,
}
agriculture_prices_dataframe = agriculture_prices_dataframe.rename(
    columns=old_to_new_column_names_dictionary)
agriculture_prices_dataframe.head(0)

,Statistic,Year,State,Unit,Value


#### Data Type Analysis Quick View

Print an analysis report of each dataset.  
- Show the top five rows of the data frame as a quick sample.
- Show the data types of each column.
- Report the count of any duplicate rows.
- Report the counts of any missing values.

In [6]:
filename: str = os.path.basename(filepath)
eda_reports.print_dataframe_analysis_report(agriculture_prices_dataframe, filename)

,Statistic,Year,State,Unit,Value
1418,"Fixed Capital Consumption - Machinery, Equipme...",2000,State,Euro Million,422.3
149,Livestock - Horses,2011,State,Euro Million,135.6
414,Crops - Wheat,2020,State,Euro Million,45.6
539,Crops - Mushrooms,2017,State,Euro Million,118.2
1013,Intermediate Consumption - Financial Intermedi...,2011,State,Euro Million,62.8


Statistic     object
Year           int64
State         object
Unit          object
Value        float64
dtype: object

,Missing,% Missing
Value,60,3.472222


In [7]:
agriculture_prices_dataframe.drop([STATE, UNIT], axis=1, inplace=True)

In [8]:
agriculture_prices_dataframe.sample()

,Statistic,Year,Value
989,Intermediate Consumption - Fertilisers,2019,578.3


### Normalizing the table

In [9]:
agriculture_prices_dataframe = agriculture_prices_dataframe.pivot_table(
    columns=STATISTIC, index=[YEAR], values=VALUE,
    dropna=True).reset_index().rename_axis(None, axis=1)

In [10]:
agriculture_prices_dataframe.sample()

,Year,Agricultural Output at Basic Prices,All Cereals,All Crops,All Livestock,All Livestock Products,All Livestock Products - Milk,All Livestock Products Other Products (excluding Milk),Compensation of Employees,Contract Work,...,Livestock - Horses,Livestock - Pig,Livestock - Poultry,Livestock - Sheep,Net Value Added at Basic Prices,Operating Surplus,Other Subsidies Less Taxes on Production,Subsidies less Taxes on Products,Subsidies on Products,Taxes on Products
19,2009,5014.0,107.0,1377.9,2222.8,1153.9,1106.5,47.4,427.7,268.7,...,172.7,300.0,115.1,159.4,43.8,1429.0,1812.9,-9.3,30.0,39.3


#### Data Type Analysis Quick View

In [11]:
eda_reports.print_dataframe_analysis_report(agriculture_prices_dataframe, filename)

,Year,Agricultural Output at Basic Prices,All Cereals,All Crops,All Livestock,All Livestock Products,All Livestock Products - Milk,All Livestock Products Other Products (excluding Milk),Compensation of Employees,Contract Work,...,Livestock - Horses,Livestock - Pig,Livestock - Poultry,Livestock - Sheep,Net Value Added at Basic Prices,Operating Surplus,Other Subsidies Less Taxes on Production,Subsidies less Taxes on Products,Subsidies on Products,Taxes on Products
30,2020,8908.3,289.5,1942.9,3591.8,2832.0,2752.7,79.2,617.5,447.0,...,216.1,601.9,180.0,303.3,2233.8,3262.8,1646.5,94.6,147.2,52.6
6,1996,6134.7,200.9,1268.7,2318.2,1573.9,1536.0,37.9,343.2,220.9,...,96.9,356.5,140.4,241.8,2518.5,2423.2,247.9,753.0,806.0,53.0
9,1999,5651.4,164.1,1184.3,2067.9,1438.1,1408.8,29.3,375.0,245.4,...,150.0,251.4,137.8,198.0,1973.9,1930.5,331.6,715.8,743.7,27.9
19,2009,5014.0,107.0,1377.9,2222.8,1153.9,1106.5,47.4,427.7,268.7,...,172.7,300.0,115.1,159.4,43.8,1429.0,1812.9,-9.3,30.0,39.3
27,2017,8476.4,237.2,1824.6,3592.4,2668.7,2594.1,74.6,525.6,379.8,...,287.4,516.8,163.1,262.9,2309.2,3422.1,1638.5,11.0,60.5,49.5


Year                                                                       int64
Agricultural Output at Basic Prices                                      float64
All Cereals                                                              float64
All Crops                                                                float64
All Livestock                                                            float64
All Livestock Products                                                   float64
All Livestock Products - Milk                                            float64
All Livestock Products Other Products (excluding Milk)                   float64
Compensation of Employees                                                float64
Contract Work                                                            float64
Crops - Barley                                                           float64
Crops - Forage Plants                                                    float64
Crops - Fresh Fruit         

,Missing,% Missing
Intermediate Consumption - Services,17,53.125
"Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)",17,53.125
Crops - Turf,7,21.875
Crops - Barley,1,3.125
Intermediate Consumption - Crop Protection Products,1,3.125
Subsidies on Products,1,3.125
Intermediate Consumption - Veterinary Expenses,1,3.125
Intermediate Consumption - Seeds,1,3.125
Intermediate Consumption - Other Goods and Services,1,3.125
Intermediate Consumption - Maintenance and Repairs,1,3.125


#### Examine the null values

In [12]:
eda_reports.print_columns_rows_with_missing_values(agriculture_prices_dataframe, [YEAR])

,Year,Crops - Barley,Crops - Fresh Fruit,Crops - Mushrooms,Crops - Oats,Crops - Other Crops,Crops - Other Fresh Vegetables,Crops - Potatoes,Crops - Sugar Beet,Crops - Turf,...,Intermediate Consumption - Crop Protection Products,Intermediate Consumption - Financial Intermediation Services Indirect,Intermediate Consumption - Maintenance and Repairs,"Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)",Intermediate Consumption - Other Goods and Services,Intermediate Consumption - Seeds,Intermediate Consumption - Services,Intermediate Consumption - Veterinary Expenses,Subsidies on Products,Taxes on Products
15,2005,67.2,34.0,110.0,6.9,51.5,89.3,78.8,66.7,32.7,...,55.9,91.0,369.4,NaN,357.9,85.5,NaN,209.5,453.4,54.2
16,2006,85.1,40.8,99.9,12.8,53.7,89.6,104.2,0.0,35.9,...,46.4,110.0,377.7,NaN,361.8,92.6,NaN,220.5,3.8,36.8
17,2007,137.7,33.0,99.6,20.6,62.7,94.8,103.9,0.0,32.8,...,50.9,111.0,393.3,NaN,370.6,105.6,NaN,225.5,0.7,41.1
18,2008,113.6,33.2,102.8,11.2,62.8,94.7,79.9,0.0,33.2,...,56.7,124.0,400.5,NaN,386.8,110.5,NaN,224.2,32.4,50.7
19,2009,63.9,32.5,98.8,9.2,57.3,108.4,86.6,0.0,34.8,...,47.6,87.0,399.7,NaN,416.3,86.7,NaN,237.0,30.0,39.3
20,2010,118.3,35.2,98.5,14.6,58.4,87.9,117.6,0.0,34.2,...,54.1,53.0,418.5,NaN,440.0,51.7,NaN,232.7,31.8,42.3
21,2011,163.2,34.9,100.2,20.7,72.1,90.7,85.5,0.0,35.0,...,58.3,62.8,428.3,NaN,450.7,56.3,NaN,250.9,31.0,42.0
22,2012,209.7,46.4,111.9,24.4,64.3,83.5,103.2,0.0,32.6,...,66.2,58.0,445.8,NaN,444.4,63.6,NaN,282.6,28.5,56.7
23,2013,207.3,48.9,121.5,24.9,60.1,87.7,164.3,0.0,0.0,...,59.0,52.7,441.4,NaN,453.5,75.5,NaN,285.1,9.4,49.8
24,2014,196.3,49.9,133.2,14.8,59.0,93.9,89.2,0.0,0.0,...,69.2,59.1,452.3,NaN,473.1,67.2,NaN,286.0,28.8,62.5


The record for the year 2021 seems to be largely incomplete. 

In [13]:
agriculture_prices1_dataframe = agriculture_prices_dataframe[(agriculture_prices_dataframe[YEAR] != 2021)]

In [14]:
eda_reports.report_missing_values(agriculture_prices1_dataframe)

,Missing,% Missing
"Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)",16,51.612903
Intermediate Consumption - Services,16,51.612903
Crops - Turf,6,19.354839


In [15]:
eda_reports.print_columns_rows_with_missing_values(agriculture_prices1_dataframe, [YEAR])

,Year,Crops - Turf,"Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)",Intermediate Consumption - Services
15,2005,32.7,NaN,NaN
16,2006,35.9,NaN,NaN
17,2007,32.8,NaN,NaN
18,2008,33.2,NaN,NaN
19,2009,34.8,NaN,NaN
20,2010,34.2,NaN,NaN
21,2011,35.0,NaN,NaN
22,2012,32.6,NaN,NaN
23,2013,0.0,NaN,NaN
24,2014,0.0,NaN,NaN


#### Data Type Analysis Quick View

In [16]:
eda_reports.print_dataframe_analysis_report(agriculture_prices1_dataframe, filename)

,Year,Agricultural Output at Basic Prices,All Cereals,All Crops,All Livestock,All Livestock Products,All Livestock Products - Milk,All Livestock Products Other Products (excluding Milk),Compensation of Employees,Contract Work,...,Livestock - Horses,Livestock - Pig,Livestock - Poultry,Livestock - Sheep,Net Value Added at Basic Prices,Operating Surplus,Other Subsidies Less Taxes on Production,Subsidies less Taxes on Products,Subsidies on Products,Taxes on Products
25,2015,7404.2,262.7,1737.3,3452.5,1949.4,1881.1,68.3,489.0,348.0,...,247.3,456.3,142.2,245.2,1666.3,2587.2,1409.9,-82.9,43.2,126.2
11,2001,6066.8,169.8,1322.5,2175.1,1604.2,1566.1,38.1,371.5,279.3,...,146.5,346.1,137.7,284.4,1979.6,2177.1,569.0,685.8,710.8,25.0
24,2014,7293.9,280.5,1747.5,3070.1,2151.3,2093.1,58.2,495.7,358.7,...,221.7,471.3,133.3,231.6,1411.1,2441.2,1525.8,-33.7,28.8,62.5
17,2007,5975.4,241.8,1632.7,2378.7,1716.1,1667.5,48.6,448.6,288.3,...,269.3,288.6,133.0,184.5,1045.8,2442.0,1844.9,-40.4,0.7,41.1
20,2010,5822.1,192.9,1670.0,2259.6,1583.9,1541.9,42.0,464.7,319.1,...,145.9,333.7,112.2,165.6,654.9,1841.5,1651.3,-10.5,31.8,42.3


Year                                                                       int64
Agricultural Output at Basic Prices                                      float64
All Cereals                                                              float64
All Crops                                                                float64
All Livestock                                                            float64
All Livestock Products                                                   float64
All Livestock Products - Milk                                            float64
All Livestock Products Other Products (excluding Milk)                   float64
Compensation of Employees                                                float64
Contract Work                                                            float64
Crops - Barley                                                           float64
Crops - Forage Plants                                                    float64
Crops - Fresh Fruit         

,Missing,% Missing
"Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)",16,51.612903
Intermediate Consumption - Services,16,51.612903
Crops - Turf,6,19.354839


### Save Artifact

Saving the output of the notebook.

In [17]:
agriculture_prices1_dataframe.to_csv(
    './../artifacts/irish-milk-production-eda-output.csv', index=None)

Author &copy; 2021 <a href="https://github.com/markcrowe-com" target="_parent">Mark Crowe</a>. All rights reserved.